In [72]:
## NLTK PREPROCESSING INPUT

## Imports

In [73]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import json
import numpy as np
import random
from collections.abc import MutableMapping
import streamlit as st


In [74]:
import collections.abc

# 👇️ add attributes to `collections` module
# before you import the package that causes the issue
collections.Mapping = collections.abc.Mapping
collections.MutableMapping = collections.abc.MutableMapping
collections.Iterable = collections.abc.Iterable
collections.MutableSet = collections.abc.MutableSet
collections.Callable = collections.abc.Callable



In [75]:
import tensorflow as tf


In [76]:
def tokenize_sentence(sentence):
    tokenize_word = word_tokenize(sentence)
    return tokenize_word


In [77]:

def bag_of_words(tokenize_sentence, all_words):
    bag = np.zeros(len(all_words), dtype = np.float32)

    for ind,word in enumerate(all_words):
        if word in tokenize_sentence:
            bag[ind] = 1.0
    return bag
       
    

In [78]:
 dic_of_words = []
def stem_sentence(sentence,stemmer= PorterStemmer()):
    
    stemmed_sentence = []
    ignore_words = ["!", "?","," , "."]
    for word in sentence:
        if word not in ignore_words:
            word = word.lower()
            dic_of_words.append(word)
            stemmed_word = stemmer.stem(word)
            stemmed_sentence.append(stemmed_word)
    return stemmed_sentence, dic_of_words
        

In [79]:
all_words = []
tags = []
xy = []
vocabulary = []


with open ("intents.json", 'r') as file:
    intents = json.load(file)
for intent in intents["intents"]:
    tag = intent["tag"]
    tags.append(tag)
    sentences = intent["patterns"]
    for items in sentences:
        w = tokenize_sentence(items)
        vocabulary.append(w)
        w, dic_of_words = stem_sentence(w)
        xy.append((w,tag))
        all_words.extend(w)


ignore_words = ["!", "?","," , "."]
set_of_word = [i.lower() for word in vocabulary for i  in word if i not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))
train_x = []
train_y = []


with open ("tags.json", 'w') as file:
    json.dump(tags, file)

with open ("all_words.json", 'w') as file:
    json.dump(all_words, file)
with open ("dic_words.json", 'w') as file:
    json.dump(dic_of_words, file)

In [80]:
print(set_of_word)

['hi', 'hiiiiii', 'hiiii', 'hey', 'is', 'anyone', 'there', 'hi', 'there', 'hello', 'hey', 'there', 'howdy', 'hola', 'bonjour', 'konnichiwa', 'guten', 'tag', 'ola', 'good', 'morning', 'good', 'afternoon', 'good', 'evening', 'good', 'night', 'bye', 'see', 'you', 'later', 'goodbye', 'au', 'revoir', 'sayonara', 'ok', 'bye', 'bye', 'then', 'fare', 'thee', 'well', 'thanks', 'thank', 'you', 'that', "'s", 'helpful', 'thanks', 'for', 'the', 'help', 'than', 'you', 'very', 'much', 'nothing', 'much', 'who', 'are', 'you', 'what', 'are', 'you', 'who', 'you', 'are', 'tell', 'me', 'more', 'about', 'yourself', 'what', 'is', 'your', 'name', 'what', 'should', 'i', 'call', 'you', 'what', "'s", 'your', 'name', 'tell', 'me', 'about', 'yourself', 'what', 'can', 'you', 'do', 'who', 'created', 'you', 'how', 'were', 'you', 'made', 'how', 'were', 'you', 'created', 'my', 'name', 'is', 'i', 'am', 'name', 'i', 'go', 'by', 'could', 'you', 'help', 'me', 'give', 'me', 'a', 'hand', 'please', 'can', 'you', 'help', 'what

In [81]:
len(tags)

80

In [82]:
for (sequence, tag )in xy:
    bag = bag_of_words(sequence, all_words)
    train_x.append(bag)

    label = tags.index(tag)
    train_y.append(label)

In [83]:
#trainy one hot encoding
hot_output = []
output_code = [0 for i in range(len(tags))]
for tag_no in train_y:
    output_tag = output_code[:]
    output_tag[tag_no] = 1
    hot_output.append(output_tag)


In [84]:
train_x = np.array(train_x)
train_y = np.array(hot_output)

In [85]:
train_x.shape

(234, 281)

In [86]:
model = tf.keras.Sequential()

In [87]:
model.add(tf.keras.layers.Dense(128, activation = "relu", input_shape =(None, train_x.shape[0], train_x.shape[1]) ))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64,activation = "relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(tags), activation = "softmax"))

In [88]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [89]:
model.fit(train_x, train_y, epochs = 200, batch_size = 50)

Epoch 1/200
5/5 [==============================] - 21s 12ms/step - loss: 4.3978 - accuracy: 0.0171
Epoch 2/200
5/5 [==============================] - 0s 7ms/step - loss: 4.3661 - accuracy: 0.0214
Epoch 3/200
5/5 [==============================] - 0s 8ms/step - loss: 4.3321 - accuracy: 0.0598
Epoch 4/200
5/5 [==============================] - 0s 18ms/step - loss: 4.3251 - accuracy: 0.0342
Epoch 5/200
5/5 [==============================] - 0s 13ms/step - loss: 4.2974 - accuracy: 0.0470
Epoch 6/200
5/5 [==============================] - 0s 9ms/step - loss: 4.2685 - accuracy: 0.0684
Epoch 7/200
5/5 [==============================] - 0s 12ms/step - loss: 4.2572 - accuracy: 0.0726
Epoch 8/200
5/5 [==============================] - 0s 9ms/step - loss: 4.2101 - accuracy: 0.0812
Epoch 9/200
5/5 [==============================] - 0s 9ms/step - loss: 4.1527 - accuracy: 0.1111
Epoch 10/200
5/5 [==============================] - 0s 11ms/step - loss: 4.1535 - accuracy: 0.1068
Epoch 11/200
5/5 [=====

In [90]:
model.save("model2.pkl")

INFO:tensorflow:Assets written to: model2.pkl\assets


INFO:tensorflow:Assets written to: model2.pkl\assets
